In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split


heart_disease = fetch_ucirepo(id=45)
X = heart_disease.data.features
Y = heart_disease.data.targets
X.fillna({'ca': X['ca'].median(), 'thal': X['thal'].mode()[0]}, inplace=True)

for index, row in Y.iterrows():
    if row[0] != 0:
        row[0] = 1 #jeżeli nie ma klasy 0 to ma klasę 1

y = Y.to_numpy()
X = (X-X.min())/(X.max()-X.min())
x = X.to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2)

x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

print(x_train)

tensor([[0.2083, 1.0000, 1.0000,  ..., 0.5000, 0.0000, 1.0000],
        [0.4583, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.5000, 1.0000, 0.6667,  ..., 1.0000, 0.0000, 0.0000],
        ...,
        [0.6875, 1.0000, 1.0000,  ..., 0.5000, 0.6667, 1.0000],
        [0.7708, 1.0000, 1.0000,  ..., 0.0000, 0.3333, 0.0000],
        [0.4583, 0.0000, 0.6667,  ..., 0.0000, 0.0000, 0.0000]])


c:\users\krzys\desktop\pythonproject\againterpreter\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [86]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(x_train.shape[1], 64)
        self.hidden_layer = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.input_layer(x)
        x = self.sigmoid(x)
        x = self.hidden_layer(x)
        x = self.sigmoid(x)
        x = self.output_layer(x)
        return x

model = NeuralNetwork()

In [87]:
# funkcja kosztu i optimizer
criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.1)
optimizer = optim.Adagrad(model.parameters(), lr=0.1)


num_epochs = 10000

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(x_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad() #bo chcemy nadpisywać gradienty a nie je akumulować
    loss.backward()
    optimizer.step()    # zmiana parametrów zgodnie z wartością gradientu

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Training complete')

Epoch [100/10000], Loss: 0.3439
Epoch [200/10000], Loss: 0.3144
Epoch [300/10000], Loss: 0.2852
Epoch [400/10000], Loss: 0.2612
Epoch [500/10000], Loss: 0.2414
Epoch [600/10000], Loss: 0.2210
Epoch [700/10000], Loss: 0.1978
Epoch [800/10000], Loss: 0.1743
Epoch [900/10000], Loss: 0.1515
Epoch [1000/10000], Loss: 0.1314
Epoch [1100/10000], Loss: 0.1141
Epoch [1200/10000], Loss: 0.0990
Epoch [1300/10000], Loss: 0.0861
Epoch [1400/10000], Loss: 0.0751
Epoch [1500/10000], Loss: 0.0657
Epoch [1600/10000], Loss: 0.0561
Epoch [1700/10000], Loss: 0.0477
Epoch [1800/10000], Loss: 0.0994
Epoch [1900/10000], Loss: 0.0382
Epoch [2000/10000], Loss: 0.0343
Epoch [2100/10000], Loss: 0.0309
Epoch [2200/10000], Loss: 0.0277
Epoch [2300/10000], Loss: 0.0247
Epoch [2400/10000], Loss: 0.0221
Epoch [2500/10000], Loss: 0.0197
Epoch [2600/10000], Loss: 0.0177
Epoch [2700/10000], Loss: 0.0159
Epoch [2800/10000], Loss: 0.0144
Epoch [2900/10000], Loss: 0.0130
Epoch [3000/10000], Loss: 0.0119
Epoch [3100/10000],

In [88]:
# ewaluacja modelu na zbiorze testowym
from sklearn.metrics import confusion_matrix
with torch.no_grad():
    test_outputs = model(x_test)
    test_outputs = (test_outputs > 0.5).float()
    accuracy = (test_outputs == y_test).sum().item() / len(y_test)
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

    cm = confusion_matrix(y_test, test_outputs)
    print("Confusion Matrix:")
    print(cm)

Test Accuracy: 77.05%
Confusion Matrix:
[[24  7]
 [ 7 23]]
